In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
model = YOLO("/Users/Moldir/Downloads/clf-results/clf_yolo_try2/weights/best.pt")

# Загружаем маску
mask_path = "/Users/Moldir/Downloads/mask_1920_1080.png"

In [4]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 16.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.1/32.1 MB 28.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1


In [5]:
import streamlit as st
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image

# Страница
st.set_page_config(page_title="Parking Spot Classification", layout="wide")
st.title("🚗 Классификация парковочных мест с помощью YOLOv8")

# Загрузка модели
@st.cache_resource
def load_model():
    model = YOLO("/Users/Moldir/Downloads/clf-results/clf_yolo_try2/weights/best.pt")  # Замени на свой путь
    return model

model = load_model()

# Загрузка видео и маски
uploaded_video = st.file_uploader("Загрузите видео парковки", type=["mp4", "avi", "mov"])
uploaded_mask = st.file_uploader("Загрузите маску парковочных мест", type=["png", "jpg"])

if uploaded_video and uploaded_mask:
    # Чтение маски
    mask = Image.open(uploaded_mask).convert('L')
    mask = np.array(mask)

    # Определение парковочных зон
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

    # Загрузка видео
    cap = cv2.VideoCapture(uploaded_video.name)

    frame_placeholder = st.empty()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        annotated_frame = frame_rgb.copy()

        for i in range(1, num_labels):
            x, y, w, h, area = stats[i]
            crop = frame_rgb[y:y+h, x:x+w]

            if crop.size == 0:
                continue

            # Предсказание
            result = model.predict(source=crop, imgsz=224, verbose=False)
            label = result[0].names[int(result[0].probs.top1)]
            confidence = float(result[0].probs.top1conf)

            color = (0, 255, 0) if label == "empty" else (255, 0, 0)
            text = f"{label} ({confidence:.2f})"

            cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(annotated_frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Отображение кадра
        frame_placeholder.image(annotated_frame, channels="RGB")

    cap.release()

else:
    st.info("Пожалуйста, загрузите видео и маску для старта 🎥🖼️")


2025-04-26 11:18:15.336 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 11:18:15.338 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 11:18:15.602 
  command:

    streamlit run /Users/Moldir/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-26 11:18:15.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 11:18:15.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 11:18:15.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 11:18:15.621 Thread 'MainThread': missing ScriptRunContext! This warning can be igno

In [ ]:
!streamlit run app.py



      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  